# Plan d'expériences de désactivation des enzymes
Valentine a réalisé un plan d'expériences. (Une série de conditions) et a enregistré les réponses (résultats de dosages))

L'enjeu de la manip est :
1. Modeliser (Trouver une équation que relie les différents paramètres étudiés et les résultats obtenus pour chaque réponse)
2. Evaluer la qualité des modèles
2. Trouver les paramètres pour lesquels on desactive les enzymes c'est-à-dire minimiser la valeur des réponses

# 0. Préliminaire

Dans un terminal (Ctrl+Maj+ù), taper:

    pip install lestpy --upgrade

Verifier que la version est bien la 0.0.8 (si j'ai le temps d'exporter mes correctifs sinon 0.0.7 mais l'analyse de sensitilité de fonctionnera pas), sinon relancer ce script

ou si ce n'est pas installé (il faut egalement avoir installé numpy, pandas, statsmodels, scipy, SALib et sklearn)

    pip install lestpy

# 1. Import des modules

+ Modules Pandas pour la gestion des tableaux
+ Modules Lestpy pour la modélisation

In [ ]:
import pandas as pd
import lestpy.lbm_model as lp

# 2. Import des données

### Obtenir le chemin d'accès au fichier de données:
Enregistrer les données à l'endroit souhaité.

Faire clique-droit sur le fichier et cliquer sur: 

    copier en tant que chemin d'accès
    

### Définition du chemin et nom du fichier de données

<!> au sens des slashs pour le chemin ! Windows met les anti-slashs \\ qu'il faut corriger en /S

In [ ]:
path, fichier = "C:/Users/TTfr/Documents/Python/Analyses DOE - MD/desactivation/","resultats DOE Val.xlsx"

### Import des données dans python avec pandas
import et visualisation des 5 premieres lignes avec .head(5)

In [ ]:
donnees = pd.read_excel(path + fichier, decimal=',')
donnees.head(5)

### Nettoyage des données et des colonnes obsolètes
la première colonne est inutile donc on la laisse tomber (.drop() ) défitinivement (inplace = True)

Et on visualise les 5 dernières lignes ( .tail(5) )

In [ ]:
donnees.drop(columns = donnees.columns[0], inplace=True)
donnees.tail(5)

### Liste des colonnes pour séparer entre X et y

In [ ]:
donnees.columns

### Définition des paramètres descriptifs X et des réponses y

et visualisation d'un descriptif de chaque tableau avec .describe()

In [ ]:
y = donnees[['X10 1','C3 1570', 'X10 1500/1', 'C1 1500/1', 'Hel1', 'C3 0614/2', 'PR1 FL', 'PR1 0120/1']]
X = donnees[['pH', 'Temperatures (°C)', 'Temps (min)', '[Acide pectique (%m/m)]']]
X.describe()

In [ ]:
y.describe()

# 3. Modelisation

### création d'un modèle par l'approche Lestpy 

In [ ]:
model = lp.LBM_Regression()

### Premier essai - Modelisation linéaire
reponse = coef1 * pH + coef2 * T°C + coef3 * temps, etc.

Les valeurs *R-squared* pour chaque modèle sont faibles (maximum à 50%) montrant que les facteurs simples ne sont pas suffisants pour expliquer les variations des réponses.

Ce résultat est plutot conforme aux attentes. En effet, les enzymes ont un pH optimal et une temperature optimale, l'activité n'augmente donc pas avec l'augmentation de la température ou du pH par exemple. 

In [ ]:
model.fit(X,y)

### Calcul de nouvelles variables 'interactions'
on calcule ces interactions en appelant la méthode .transform(X,y)


In [ ]:
model.transform(X, y)

La liste affichée *['X_xor_Y', 'X_or_Y'...]* correspond aux interactions utilisées.


Le tableau est maintenant composé des facteurs simples mais aussi d'interactions entre ceux-ci

### Second essai - modélisation avec interactions
Les valeurs *R-squared* sont maintenant bien meilleures. A une exception près (modele 'C3 1570') les valeurs sont supérieures à 85% 

In [ ]:
model.fit()

#### On peut afficher l'expression exactes des différents modèles avec la methode .print_model()

In [ ]:
model.print_model()

#### On peut avoir un résumé des scores d'ajustement des différents modèles avec la methode .fitting_score()

In [ ]:
for i in y:
    model.fitting_score(y[i])

# 4. Visualisation des différentes modélisations

On visualisera différents élements en créant une instance de la classe Display() et en passant notre modèle en paramètre

In [ ]:
graf = lp.Display(model)

## 4.1 Une première méthode consiste à imprimer un descriptif des modèles.
Le descriptif comprend:
1. les scores d'ajustement
2. le graph d'ajustement (y modélisé en fonction de y mesuré)
3. la distribution des résidus
4. Le graphique d'optimisation du R2 de prédiction = Q2 en fonction du nombre de facteurs dans le modèle (l'idée etant de maximiser le Q2 et minimiser le nombre de facteurs)

In [ ]:
graf.describe()

## 4.2 Domaine experimental
On décrit d'abord le domaine expérimental (= le cadre) dans lequel aura lieu l'optimisation (quelle gamme de pH, de temp, etc.)
On a accès à une méthode automatisée .feature_analysis() qui renvoie le domaine expérimental analysée, le mélange (mix) si il y a un mélange

domain est un dictionnaire dont les clés sont les facteurs et les valeurs sont une liste comprenant :
+ [statut (inutile à ce stage), minimum, maximum, [valeurs si variable], type de variable (discrète ou continue)]

In [ ]:
domain, mix = model.features_analysis(X)

## 4.3 Analyse de sensibilité
L'objectif est de voir à quel point les variation des variables seules ou en interactions sont impactantes sur la valeur finale du modèle

In [ ]:
graf.sensibility_analysis(domain)

## 5. Optimisation

L'objectif de cette manip est de trouver les conditions idéales selon le modèle pour minimiser l'activité des enzymes donc maximiser la desactivation

In [ ]:
domain, mix = model.features_analysis(X)
print(domain)

## 5.2 Definition des objectifs
Les objectifs sont une liste qui décrit pour chaque variable dans l'ordre l'objectif d'optimisation que l'on souhaite parmi:
minimisation, maximisation, valeur à atteindre ou aucun objectif

Ici on souhaite minimiser toutes les réponses donc on crée une liste de 8 'minimize' car il y a 8 modèles

In [ ]:
target = ['minimize'] * 8
print(target)

## 5.3 Minimisation globale
On lance l'optimisation avec la méthode .optimize() dans laquelle on injecte le domaine experimental et les objectifs.

In [ ]:
model.optimize(domain, target)

On obtient deux tableaux:
1. Le premier correspond aux résulats (la condition la meilleure pour atteindre **tous les objectifs en même temps**). La désirabilité montre le score obtenu sur **ces conditions pour toutes les réponses**
3. Le second est l'ensemble des essais menés pour obtenir ces résultats (obsolete)

## 5.4 Maximisation globale
On peut faire le meme exercice pour maximiser les valeurs et donc voir les conditions optimales pour **toutes les activités en même temps**

(en laissant les objectifs à None on maximise tout)

In [ ]:
maximiser = ['maximize']*8
model.optimize(domain, maximiser)
